In [26]:
import googleapiclient.discovery
import requests
import csv

In [27]:
# Set up the YouTube API client
api_key = ""
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

# Specify the channel custom URL
channel_custom_url = "https://www.youtube.com/@yougotfam"

# Extract the channel username from the custom URL
channel_username = channel_custom_url.split("/")[-1]

# Search for the channel by username
search_response = youtube.search().list(part="id", q=channel_username, type="channel", maxResults=1).execute()
channel_id = search_response["items"][0]["id"]["channelId"]

In [28]:
# retrieve the channel's metadata
channel = youtube.channels().list(part="snippet,statistics", id=channel_id).execute()["items"][0]
print("Channel name:", channel["snippet"]["title"])
print("Channel description:", channel["snippet"]["description"])
print("Number of subscribers:", channel["statistics"]["subscriberCount"])
print("Total number of views:", channel["statistics"]["viewCount"])


Channel name: Fam India
Channel description: FamX by Fam, is India’s first Spending Account made for everyone.

Number of subscribers: 56300
Total number of views: 2612678


In [29]:
# retrieve the channel's videos and their metadata
videos = []
next_page_token = None
while True:
    request = youtube.search().list(part="id", channelId=channel_id, maxResults=50, pageToken=next_page_token, type="video")
    response = request.execute()
    video_ids = [item["id"]["videoId"] for item in response["items"]]
    video_request = youtube.videos().list(part="snippet,statistics", id=",".join(video_ids))
    video_response = video_request.execute()
    videos += video_response["items"]
    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break

In [30]:
with open("video_data.csv", "w", newline='', encoding='utf-8') as csvfile:
    fieldnames = ["video_title", "video_id", "video_url", "views", "likes", "comments", "caption", "upload_date"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Retrieve captions and video data, and save them to the CSV file
    for video in videos:
        video_data = {}
        video_data["video_id"] = video["id"]
        video_data["video_title"] = video["snippet"]["title"]
        video_data["video_url"] = f"https://www.youtube.com/watch?v={video_data['video_id']}"
        if "statistics" in video:
            video_data["views"] = video["statistics"].get("viewCount", "N/A")
            video_data["likes"] = video["statistics"].get("likeCount", "N/A")
            # video_data["dislikes"] = video["statistics"].get("dislikeCount", "N/A")
            video_data["comments"] = video["statistics"].get("commentCount", "N/A")
            video_data["upload_date"] = video["snippet"]["publishedAt"]

        else:
            video_data["views"] = "N/A"
            video_data["likes"] = "N/A"
            # video_data["dislikes"] = "N/A"
            video_data["comments"] = "N/A"
            video_data["upload_date"] = "N/A"
        
        caption_request = youtube.captions().list(part="id", videoId=video_data["video_id"])
        caption_response = caption_request.execute()
        if caption_response["items"]:
            caption_id = caption_response["items"][0]["id"]
            caption_download_url = f"https://www.youtube.com/api/timedtext?v={video_data['video_id']}&lang=en&fmt=srv3"
            # download the caption file and save the text
            caption_file = requests.get(caption_download_url).text
            video_data["caption"] = caption_file

        else:
            video_data["caption"] = "No caption available."
        
        # Write the data to the CSV file
        writer.writerow(video_data)

        # Print the data to the console
        print("Video title:", video_data["video_title"])
        print("Video ID:", video_data["video_id"])
        print("Video URL:", video_data["video_url"])
        print("Views:", video_data["views"])
        print("Likes:", video_data["likes"])
        print("Dislikes:", video_data["upload_date"])
        print("Comments:", video_data["comments"])
        print("Caption:\n", video_data["caption"])

Video title: Happy Diwali 2020 | FamPay
Video ID: hiQ4nLVbqY4
Video URL: https://www.youtube.com/watch?v=hiQ4nLVbqY4
Views: 2497
Likes: 156
Dislikes: 2020-11-14T05:05:55Z
Comments: N/A
Caption:
 No caption available.
Video title: Get Featured In Tanmay Bhat’s Next Video #shorts
Video ID: YEK2T3YLT9M
Video URL: https://www.youtube.com/watch?v=YEK2T3YLT9M
Views: 14725
Likes: 786
Dislikes: 2021-03-11T03:20:57Z
Comments: N/A
Caption:
 No caption available.
Video title: FamPay Explained To Parents | All about India's First Neobank for Teens
Video ID: BTcwwlx5Xqk
Video URL: https://www.youtube.com/watch?v=BTcwwlx5Xqk
Views: 371303
Likes: 11755
Dislikes: 2021-01-22T12:25:56Z
Comments: N/A
Caption:
 No caption available.
Video title: WE USED A METAL DETECTOR AND FOUND THIS 😳🤯
Video ID: X9gwJqLJ6nE
Video URL: https://www.youtube.com/watch?v=X9gwJqLJ6nE
Views: 4272
Likes: N/A
Dislikes: 2022-10-20T06:43:15Z
Comments: 14
Caption:
 No caption available.
Video title: How to activate your FamCard: st